<a href="https://www.kaggle.com/code/asad11914/orange-diseases-detection?scriptVersionId=169759593" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

# Load dataset from directory to Tensor

In [ ]:
#Path ke folder dataset
dir_path = '/kaggle/input/orange-disease-parted/Orange_disease_90-10/dataset/'

#membuat dataset untuk train, validation, dan testing
train_ds = keras.utils.image_dataset_from_directory(
    dir_path + 'train/', 
    labels= "inferred",
    validation_split= 0.2,
    subset= "training",
    seed= 100,
    batch_size= None
)

val_ds = keras.utils.image_dataset_from_directory(
    dir_path + 'train/', 
    labels= "inferred",
    validation_split= 0.2,
    subset= "validation",
    seed= 100,
    batch_size= None
)

test_ds = keras.utils.image_dataset_from_directory(
    dir_path + 'test/',
    labels= "inferred",
    batch_size= None
)

# Preprocess Data

In [ ]:
def process_images(image, label):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image, (227,227))
    return image, label

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(val_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

In [ ]:
training_ds = (train_ds
            .shuffle(train_ds_size)
            .map(process_images)
            .batch(batch_size=32, drop_remainder=True))
testing_ds = (test_ds
           .shuffle(test_ds_size)
           .map(process_images)
           .batch(batch_size=32, drop_remainder=True))
validation_ds = (val_ds
          .shuffle(validation_ds_size)
          .map(process_images)
          .batch(batch_size=32, drop_remainder=True))

# Create AlexNet Model

In [ ]:
tf.config.run_functions_eagerly(True)
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])

In [ ]:
root_logdir = os.path.join(os.curdir, "logs/fit/")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer= keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'],
    )
model.summary()

In [ ]:
history = model.fit(
    training_ds,
    epochs=50,
    validation_data=validation_ds,
    validation_freq=1,
    callbacks= [tensorboard_cb]
)

In [ ]:
plt.title("Akurasi Training 90%-10%")
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.3, 1])
plt.legend(loc='lower right')

In [ ]:
model.evaluate(testing_ds)

In [ ]:
model.save("model.h5")